In [18]:
import os
from dotenv import load_dotenv
load_dotenv()
SUMMARIZE_MODEL_OCID = os.getenv('SUMMARIZE_MODEL_OCID') 
GEN_AI_INFERENCE_ENDPOINT = os.getenv('GEN_AI_INFERENCE_ENDPOINT') 
COMPARTMENT_ID = os.getenv('COMPARTMENT_ID') 
GENERATION_MODEL_OCID = os.getenv('GENERATION_MODEL_OCID') 
GEN_AI_ENDPOINT = os.getenv('GEN_AI_ENDPOINT') 
GENERATION_MODEL_OCID_llam = os.getenv('GENERATION_MODEL_OCID_llam') 

import oci
config = oci.config.from_file()

In [19]:
import textwrap

# Function to wrap text at 80 characters per line while keeping sentences intact
def wrap_text_by_sentence(text, target_language_code, width=80):
    if target_language_code == 'ja':
        splitStr = "。" # Split text into sentences based on period (。)
    else:
        splitStr = ". "
    sentences = text.split(splitStr) 
    wrapped_sentences = []
    for sentence in sentences:
        if sentence:
            # Wrap each sentence with a width of 80 characters
            wrapped_sentence = textwrap.fill(sentence + splitStr, width=width)
            wrapped_sentences.append(wrapped_sentence)
    return "\n".join(wrapped_sentences)

## translate with OCI

In [30]:

# translate methord made with ai_language
def translate_and_wrap_text(text_from, compartment_id, source_language_code, target_language_code):
    # Initialize service client with default config file
    ai_language_client = oci.ai_language.AIServiceLanguageClient(config)
    
    # Send the request to service, some parameters are not required, see API doc for more info
    batch_language_translation_response = ai_language_client.batch_language_translation(
        batch_language_translation_details=oci.ai_language.models.BatchLanguageTranslationDetails(
            documents=[
                oci.ai_language.models.TextDocument(
                    key="1",
                    text=text_from,
                    language_code=source_language_code)],
            compartment_id=compartment_id,
            target_language_code=target_language_code),
        )
    
    # Get the data from response
    translated_text = batch_language_translation_response.data.documents[0].translated_text
    
    # Wrap the translated text by sentence
    wrapped_text = wrap_text_by_sentence(translated_text,target_language_code)
    
    return wrapped_text



In [31]:

# Example usage
text_from = "Proper waste management is essential to the health of all living systems. Our grantees include Wecyclers, a social enterprise in Lagos, Nigeria, that incentives waste recycling in densely populated urban neighbourhoods, and the WEEE Centre in Nairobi, Kenya, which promotes public awareness of the environmental and health hazards of e-waste and educates the public about reuse, recycling, and safe disposal. Oracle is also a founding sponsor of California Coastal Cleanup Day, which has removed hundreds of thousands of pounds of trash from California waterways each year since 1995"
source_language_code = "en"
target_language_code = "ja"

translated_and_wrapped_text = translate_and_wrap_text(text_from, COMPARTMENT_ID, source_language_code, target_language_code)
print(translated_and_wrapped_text)

適切な廃棄物管理は、すべての生活システムの健全性に不可欠です。
私たちの助成金には、ナイジェリアのラゴスの社会企業であるウェサイクラーズ、人口密集した都市部での廃棄物リサイクルを奨励するウェサイクラーズ、ケニアのナイロビのW
EEEセンター、電子廃棄物の環境および健康上の危険に対する公衆の認識を促進し、再利用、リサイクル、および安全な処分について市民に教育する。
OracleはCalifornia Coastal Cleanup
Dayの創設スポンサーでもあり、1995年以来、毎年何十万ポンドものゴミをCalifornia Waterwaysから削除しています。


In [32]:
# Example usage
text_from = "適切な廃棄物管理は、すべての生活システムの健全性に不可欠です。\n私たちの助成金には、ナイジェリアのラゴスの社会企業であるウェサイクラーズ、人口密集した都市部での廃棄物リサイクルを奨励するウェサイクラーズ、ケニアのナイロビのWEEEセンター、電子廃棄物の環境および健康上の危険に対する公衆の認識を促進し、再利用、リサイクル、および安全な処分について市民に教育する。"
source_language_code = "ja"
target_language_code = "en"

translated_and_wrapped_text = translate_and_wrap_text(text_from, COMPARTMENT_ID, source_language_code, target_language_code)
print(translated_and_wrapped_text)

Proper waste management is essential to the health of all living systems.
Our grants include Wesaiklers, a social enterprise in Lagos, Nigeria, Wesaiklers
to encourage waste recycling in densely populated urban areas, and WEEE Center
in Nairobi, Kenya.
Promote public awareness of the environmental and health hazards of e-waste and
educate citizens about reuse, recycling and safe disposal.


##  generate with cohere or meta.llama-2-70b-chat

In [23]:
class GenerativeAiClient:
    def __init__(self, config, service_endpoint):
        self.gen_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
            config=config,
            service_endpoint=service_endpoint,
            retry_strategy=oci.retry.NoneRetryStrategy(),
            timeout=(10, 240)
        )
        self.generate_text_details = oci.generative_ai_inference.models.GenerateTextDetails()
        self.inference_requests = oci.generative_ai_inference.models.CohereLlmInferenceRequest()
        self.llam_inference_request = oci.generative_ai_inference.models.LlamaLlmInferenceRequest()

    def generate_job_description_cohere(self, prompt, compartment_id, model_id, parameters=None):
      
        # Set parameters values
        self.inference_requests.prompt = prompt
        self.inference_requests.max_tokens = parameters.get('max_tokens', 600)
        self.inference_requests.temperature = parameters.get('temperature', 0.75)
        self.inference_requests.is_stream = parameters.get('is_stream', False)
        self.inference_requests.num_generations = parameters.get('num_generations', 1)

        self.generate_text_details.compartment_id = compartment_id
        self.generate_text_details.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id=model_id)
        self.generate_text_details.inference_request = self.inference_requests

        generate_text_response = self.gen_ai_inference_client.generate_text(self.generate_text_details)

        return generate_text_response.data
    
    def generate_job_description_llam(self, llam_model_ocid, compartment_id, prompt, parameters=None):
        generate_text_detail = oci.generative_ai_inference.models.GenerateTextDetails()
        
        self.llam_inference_request.prompt = prompt

        # Set parameters values
        self.llam_inference_request.max_tokens = parameters.get('max_tokens', 600)
        self.llam_inference_request.temperature = parameters.get('temperature', 1)
        self.llam_inference_request.frequency_penalty = parameters.get('frequency_penalty', 0)
        self.llam_inference_request.top_p = parameters.get('top_p', 0.75)

        generate_text_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id=llam_model_ocid)
        generate_text_detail.inference_request = self.llam_inference_request
        generate_text_detail.compartment_id = compartment_id

        generate_text_response = self.gen_ai_inference_client.generate_text(generate_text_detail)

        return generate_text_response.data


In [50]:
# Create an instance of GenerativeAiClient
client = GenerativeAiClient(config=config, service_endpoint=GEN_AI_INFERENCE_ENDPOINT)

# Using the second method to generate job description
compartment_id = COMPARTMENT_ID
prompt = """
Generate a job description for a data visualization expert with the following three qualifications only:
1) At least 5 years of data visualization experience
2) A great eye for detail
3) Ability to create original visualizations
"""
model_ocid = GENERATION_MODEL_OCID

# Define parameters to adjust
parameters = {
    'max_tokens': 800
    #,'num_generations': 3
}

# Generate job description using generate_job_description_cohere method and adjust parameters here
generated_text_cohere = client.generate_job_description_cohere(prompt, compartment_id,  model_ocid, parameters)

In [51]:
print("**************************Generate Texts Result**************************")
generated_result = generated_text_cohere.inference_response.generated_texts[0].text
# Wrap the translated text by sentence
generated_result_wrapped_text = wrap_text_by_sentence(generated_result,'en')
print(generated_result_wrapped_text)

**************************Generate Texts Result**************************
 We're looking for a talented Data Visualization Expert to join our team! The
ideal candidate will have at least 5 years of experience in data visualization,
a strong eye for detail, and the ability to create innovative and original
visualizations that effectively communicate complex data insights to diverse
audiences.
  The Data Visualization Expert will play a crucial role in designing and
developing visually appealing and informative representations of data for our
company.
They will work closely with our team of data analysts and scientists to
understand the findings and insights from complex data sets and transform them
into concise and easy-to-understand visualizations.
  Key responsibilities include: - Designing and developing high-quality
visualizations that effectively communicate data insights to stakeholders,
including executives, clients, and team members across different departments. -
Interpreting d

In [52]:
# Create an instance of GenerativeAiClient
client = GenerativeAiClient(config=config, service_endpoint=GEN_AI_INFERENCE_ENDPOINT)

# Using the second method to generate job description
llam_model_ocid = GENERATION_MODEL_OCID_llam
endpoint = GEN_AI_INFERENCE_ENDPOINT
compartment_id = COMPARTMENT_ID
prompt = """
Generate a job description for a data visualization expert with the following three qualifications only:
1) At least 5 years of data visualization experience
2) A great eye for detail
3) Ability to create original visualizations
"""

In [53]:
# Define parameters to adjust
parameters = {
    'max_tokens': 800,
    'temperature': 1.2,
    'frequency_penalty': 0.1,
    'top_p': 0.8
}

# Generate job description using generate_job_description_llam method and adjust parameters here
generated_text = client.generate_job_description_llam(llam_model_ocid, compartment_id, prompt, parameters)

In [54]:

# Process the generated job description
print("**************************Generate Texts Result**************************")
generated_result = generated_text.inference_response.choices[0].text
# Wrap the translated text by sentence
generated_result_wrapped_text = wrap_text_by_sentence(generated_result,'en')
print(generated_result_wrapped_text)

**************************Generate Texts Result**************************
 Job Title: Data Visualization Expert  Job Summary:  We are seeking a highly
skilled Data Visualization Expert to join our team.
The successful candidate will have at least 5 years of experience in data
visualization, a great eye for detail, and the ability to create original
visualizations.
The Data Visualization Expert will be responsible for creating interactive and
informative visualizations that help our clients understand complex data and
make informed decisions.
This is a full-time position located in our downtown office.  Responsibilities:
* Create interactive and informative data visualizations that effectively
communicate complex data insights to clients * Work closely with clients to
understand their data and visualization needs * Develop and maintain databases
and data systems to support data visualization efforts * Design and implement
data visualization tools and systems * Collaborate with data scie

## processing flow with only generate

In [24]:
prompt = """
データ可視化の専門家のための仕事の説明を、次の3つの要件のみで生成してください：
1) 少なくとも5年のデータ可視化の経験があること
2) 細部に優れた目を持っていること
3) オリジナルの可視化を作成できる能力があること
"""

In [25]:
# Example usage
text_from = prompt
source_language_code = "ja"
target_language_code = "en"

translated_and_wrapped_text = translate_and_wrap_text(text_from, COMPARTMENT_ID, source_language_code, target_language_code)

In [26]:
# Create an instance of GenerativeAiClient
client = GenerativeAiClient(config=config, service_endpoint=GEN_AI_INFERENCE_ENDPOINT)

# Using the second method to generate job description
compartment_id = COMPARTMENT_ID
model_ocid = GENERATION_MODEL_OCID

# Define parameters to adjust
parameters = {
    'max_tokens': 800
    #,'num_generations': 3
}

# Generate job description using generate_job_description_cohere method and adjust parameters here
generated_text_cohere = client.generate_job_description_cohere(translated_and_wrapped_text, compartment_id,  model_ocid, parameters)
generated_result = generated_text_cohere.inference_response.generated_texts[0].text

In [17]:
text_from = generated_result
source_language_code = "en"
target_language_code = "ja"

translated_text = translate_and_wrap_text(text_from, COMPARTMENT_ID, source_language_code, target_language_code)

# Wrap the translated text by sentence
wrapped_text = wrap_text_by_sentence(translated_text,target_language_code)
    
print(wrapped_text)



 肩書: シニア・データ・ビジュアライゼーション・スペシャリスト

職務内容:
高度なスキルを持つ経験豊富なシニア・データ・ビジュアライゼーション・スペシャリストがチームに加わることを目指しています
理想的な候補者は、複雑なデータを魅力的で洞察力のある視覚表現に変換するための少なくとも5年間の専門知識を持つことになります
シニア・データ・ビジュアライゼーション・スペシャリストとして、データ主導型の意思決定と通信の有効性を確保するうえで重要な役割を果たします
作業によって、組織全体の利害関係者がデータをより簡単に理解して解釈できるようになり、より適切な情報に基づいたアクションと結果につながります
職責:
1.複雑な情報を技術者と非技術者の両方に効果的に伝達する、高品質なデータ・ビジュアライゼーションを設計および開発します

2.データ・アナリスト、科学者、その他の利害関係者と協力して、データのニーズと要件を理解し、適切なインサイトを効果的に伝えるようにビジュアライゼーションを調整します

3.様々なビジュアライゼーション・タイプ、手法およびツール(チャート、グラフ、マップ、インフォグラフィックなど)を試して、データに最適なビジュアル表現を判断します

4.すべてのデータ・ビジュアライゼーションで一貫したビジュアル言語を維持するために、ブランディングとデザインのガイドラインを遵守します

5.調査を実施し、業界動向やデータ視覚化の最新ツールを常に最新の状態に保ち、実践を継続的に改善します
要件:
1.データ・ビジュアライゼーションのプロフェッショナルな経験が5年以上あり、ビジュアル・クリエーションの強力なポートフォリオが実証されています

2.データ視覚化テクノロジ(RやPythonなど)やデータ分析ツール(SQLなど)で一般的に使用されるプログラミング言語の熟達度

3.多様なオーディエンスに共鳴するオリジナルの効果的なビジュアライゼーションを作成し、倫理的なデータ・プラクティスの正確性と順守を確保する機能
オファー: - 優れた可視性: 私たちはあなたの専門知識を評価し、あなたの貢献を認識することの重要性を理解しています
この役割では、人材を紹介し、データ・ビジュアライゼーションの取組みにおいて重要なプレーヤーとして認められる機会があります
- オリジナルのビジ